In [4]:
import numpy as np
import numpy.matlib

In [4]:
# Define uma matriz A, base para os cálculos posteriores

A = np.matrix([[3.,2.,4.],[1.,1.,2.],[4.,3.,2.]])
print(A)

[[ 3.  2.  4.]
 [ 1.  1.  2.]
 [ 4.  3.  2.]]


In [16]:
# Determinante de A:

x = np.linalg.det(A)
print(x)

if x != 0:
    print('true')

-4.0
true


In [17]:
# Cria a matriz M0 a partir de uma matriz A qualquer

def matrizM0(A):
    M0 = np.zeros((3,3))
    M0[1,0] = - A[1,0]/A[0,0]
    M0[2,0] = - A[2,0]/A[0,0]
    for i in range(3): M0[i,i] = 1
    return M0

M0 = matrizM0(A)
print(M0)

[[ 1.          0.          0.        ]
 [-0.33333333  1.          0.        ]
 [-1.33333333  0.          1.        ]]


In [18]:
# Obtém uma matriz A1 a partir de A. Utiliza a função matriz M0, definida no item anterior

def matrizA1(A):
    M0 = matrizM0(A)
    A1 = M0*A
    return A1

A1 = matrizA1(A)
print(A1)

[[ 3.          2.          4.        ]
 [ 0.          0.33333333  0.66666667]
 [ 0.          0.33333333 -3.33333333]]


In [19]:
# Cria a matriz M1 a partir de uma matriz A qualquer. Utiliza a função matrizA1, definida no item anterior

def matrizM1(A):
    M1 = np.zeros((3,3))
    A1 = matrizA1(A)
    M1[2,1] = - A1[2,1]/A1[1,1]
    for i in range(3): M1[i,i] = 1
    return M1

M1 = matrizM1(A)
print(M1)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0. -1.  1.]]


In [20]:
# Obtém uma matriz A2 a partir de A. Utiliza a função matrizM1, definida no item anterior

def matrizA2(A):
    M1 = matrizM1(A)
    A2 = M1*A1
    return A2

A2 = matrizA2(A)
print(A2)

[[  3.00000000e+00   2.00000000e+00   4.00000000e+00]
 [  0.00000000e+00   3.33333333e-01   6.66666667e-01]
 [  0.00000000e+00  -5.55111512e-17  -4.00000000e+00]]


In [113]:
# Obtém a matriz b2 de resultados a partir da matriz de coeficientes A e da matriz de resultados b. Utiliza as funções
# matrizM0 e matrizM1, definidas anteriormente

def matrizB2(A, b):
    M0 = matrizM0(A)
    M1 = matrizM1(A)
    b1 = M0*b
    b2 = M1*b1
    return b2

b = np.matrix([[1.],[2.],[3.]])
b2 = matrizB2(A,b)
print(b2)

[[  1.00000000e+00]
 [  1.66666667e+00]
 [ -7.40184823e-16]]


In [14]:
# Dado um sistema S, cuja solução é Determinada por Ax = b, calcula os fatores L e U e obtém a solução geral

def fatoracaoLU(A,b):
    L = np.zeros((A.shape[0],A.shape[0]))
    for i in range(A.shape[0]):
        L[i,i] = 1
    U = A
    for i in range(A.shape[0]-1):
        for j in range(i+1,A.shape[0]):
            L[j,i] = U[j,i]/U[i,i]
            for k in range(i, A.shape[0]):
                U[j,k] = U[j,k] - L[j,i]*U[i,k]
            b[j,0] = b[j,0] - L[j,i]*b[i,0]
    return L, U, b

def solucaoDoSistema(A,b):
    L = fatoracaoLU(A,b)[0]
    U = fatoracaoLU(A,b)[1]
    R = fatoracaoLU(A,b)[2]
    
    y = np.zeros((A.shape[0],1))    
    y[0,0] = R[0,0]/L[0,0]    
    for i in range(1, A.shape[0]):
        s = 0
        for j in range(0,i):
            s = s + L[j,i]*y[j,0]
            y[i,0] = (R[i,0] - s)/L[i,i]
    
    x = np.zeros((A.shape[0],1))
    x[A.shape[0]-1,0] = y[A.shape[0]-1,0]/U[A.shape[0]-1,A.shape[0]-1]
    for i in range(A.shape[0]-1,-1,-1):
        s = 0
        for j in range(i+1,A.shape[0]):
            s = s + U[i,j]*x[j,0]
            x[i,0] = (y[i,0] - s)/U[i,i]
    
    return y,x

F = np.matrix([[3.,2.,4.],[1.,1.,2.],[4.,3.,2.]])
G = np.matrix([[1.],[2.],[3.]])
print("Ly = b.\n y = \n", solucaoDoSistema(F,G)[0])
print("\nUx = y.\n x = \n", solucaoDoSistema(F,G)[1])

Ly = b.
 y = 
 [[  1.00000000e+00]
 [  1.66666667e+00]
 [ -7.21644966e-16]]

Ux = y.
 x = 
 [[ -3.00000000e+00]
 [  5.00000000e+00]
 [  1.80411242e-16]]


In [85]:
# Aplica a fatoração LU utilizando estratégias de pivoteamento parcial. Retorna a solução do sistema

def fatoracaoLU_comPivoteamento(A,b):
    p = []
    c = np.zeros((A.shape[0],1), dtype = 'float')
    y = np.zeros((A.shape[0],1), dtype = 'float')
    x = np.zeros((A.shape[0],1), dtype = 'float')
  
    for i in range(A.shape[0]):
        p.append(i)
    
    for k in range(A.shape[0]-1):
        pv = abs(A[k,k])
        r = k
        
        for i in range(k+1, A.shape[0]):
            if abs(A[i,k]) > pv:
                pv = A[i,k]
                r = i
        
        if pv == 0: break

        if r != k:
            aux = p[k]
            p[k] = p[r]
            p[r] = aux
            
            for j in range(A.shape[0]):
                aux = A[k,j]
                A[k,j] = A[r,j]
                A[r,j] = aux
       
        for i in range(k+1, A.shape[0]):
            m = A[i,k]/A[k,k]
            A[i,k] = A[i,k] - m*A[k,k]
            for j in range(k+1, A.shape[0]):
                A[i,j] = A[i,j] - m*A[k,j]
            b[i,0] = b[i,0] - m*b[k,0]
            
    for i in range(A.shape[0]):
        r = p[i]
        c[i,0] = b[r,0]
    
    for i in range(A.shape[0]):
        soma = 0
        for j in range(i-1):
            soma = soma + A[i,j]*y[j,0]
        y[i,0] = c[i,0] - soma

    for i in range(A.shape[0]-1,-1,-1):
        soma = 0
        for j in range(i+1, A.shape[0]):
            soma = soma + A[i,j]*x[j,0]
        x[i,0] = (y[i,0] - soma)/A[i,i]
   
    return x

H = np.matrix([[3.,2.,4.],[1.,1.,2.],[4.,3.,2.]])
I = np.matrix([[1.],[2.],[3.]])

print("Solução =\n", fatoracaoLU_comPivoteamento(H,I))

Solução =
 [[-3.25]
 [ 5.5 ]
 [ 0.25]]
